In [1]:
pip install bs4

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install requests

In [51]:
from bs4 import BeautifulSoup
import requests
from faker import Faker
import json  
import os

In [52]:
def get_soup_retry(url):
    fake = Faker()
    uag_random = fake.user_agent()

    header = {
        'User-Agent': uag_random,
        'Accept-Language': 'en-US,en;q=0.9'
    }
    isCaptcha = True
    while isCaptcha:
        page = requests.get(url, headers=header)
#         print("PAGE STATUS CODE", page.status_code)
        assert page.status_code == 200
        soup = BeautifulSoup(page.content, 'html.parser')
        if 'captcha' in str(soup):
            uag_random = fake.user_agent()
#             print(f'\rBot has been detected... retrying ... use new identity: {uag_random} ', end='', flush=True)
            continue
        else:
#             print('Bot bypassed')
            return soup

In [53]:
# get input product link
URL = input("Enter product URL")
soup = get_soup_retry(URL)
# print(soup)


Enter product URLhttps://www.amazon.in/Bacca-Bucci-Black-Orange-Running/dp/B08HNH5Q9S/ref=sr_1_1_sspa?crid=3GWBIZ74JJ9H6&keywords=shoes&qid=1689231831&s=shoes&sprefix=shoe%2Cshoes%2C215&sr=1-1-spons&sp_csd=d2lkZ2V0TmFtZT1zcF9hdGY&th=1&psc=1


In [54]:
price = soup.find("span", attrs={"class":'a-offscreen'}).get_text().strip()
# print(price)

In [55]:
title = soup.find("span", attrs={"id":'productTitle'}).text.strip()
# print(title)

In [56]:
total_review_count = soup.find("div", attrs={"data-hook":'total-review-count'}).text.strip()
# print(total_review_count)

In [57]:
specs_obj={}
specs = soup.find_all("tr",{"class":"a-spacing-small"})
for u in range(0,len(specs)):
    spanTags = specs[u].find_all("span")
    specs_obj[spanTags[0].text]=spanTags[1].text
    print(spanTags[0].text, spanTags[1].text)
# print(specs_obj)    

In [58]:
about=[]
specs = soup.find("ul",{"class":"a-unordered-list a-vertical a-spacing-mini"})
spanTags = specs.find_all("span")
for u in range(0,len(spanTags)):
    about.append(spanTags[u].text)
# print(about)    

In [59]:
link=[]
for i in soup.findAll("a",{'data-hook':"see-all-reviews-link-foot"}):
        link.append(i['href'])
# print(link)

In [60]:
def Searchreviews(review_link):
    url="https://www.amazon.in"+review_link
#     print(url)
    return get_soup_retry(url)

In [61]:
reviews=""
for j in range(len(link)):
    for k in range(1,2):
        new_soup=Searchreviews(link[j]+'&pageNumber='+str(k))
        for i in new_soup.findAll("span",{'data-hook':"review-body"}):
            reviews = reviews + os.linesep + os.linesep + (i.text.strip())
        if not new_soup.find('li', {'class': 'a-disabled a-last'}):
            pass
        else:
            break 
# print(reviews)

In [62]:
  save_file = open("product_reviews.txt", "w")  
save_file.write("Title: "+ title + os.linesep + os.linesep)
save_file.close() 
file = open("product_reviews.txt", "a")
file.write("Price: "+ price + os.linesep + os.linesep)
file.write("Specs: "+json.dumps(specs_obj) + os.linesep + os.linesep)
file.write("About Item: "+json.dumps(about) + os.linesep + os.linesep)
file.write("Total Review Count: "+ total_review_count + os.linesep + os.linesep)
file.write("Reviews:" + reviews)
file.close()